# Linear Regression Consulting Project

Congratulations! You've been contracted by Hyundai Heavy Industries to help them build a predictive model for some ships. [Hyundai Heavy Industries](http://www.hyundai.eu/en) is one of the world's largest ship manufacturing companies and builds cruise liners.

You've been flown to their headquarters in Ulsan, South Korea to help them give accurate estimates of how many crew members a ship will require.

They are currently building new ships for some customers and want you to create a model and use it to predict how many crew members the ships will need.

Here is what the data looks like so far:

    Description: Measurements of ship size, capacity, crew, and age for 158 cruise
    ships.


    Variables/Columns
    Ship Name     1-20
    Cruise Line   21-40
    Age (as of 2013)   46-48
    Tonnage (1000s of tons)   50-56
    passengers (100s)   58-64
    Length (100s of feet)  66-72
    Cabins  (100s)   74-80
    Passenger Density   82-88
    Crew  (100s)   90-96
    
It is saved in a csv file for you called "cruise_ship_info.csv". Your job is to create a regression model that will help predict how many crew members will be needed for future ships. The client also mentioned that they have found that particular cruise lines will differ in acceptable crew counts, so it is most likely an important feature to include in your analysis! 

Once you've created the model and tested it for a quick check on how well you can expect it to perform, make sure you take a look at why it performs so well!

In [1]:
from pyspark.sql import SparkSession
from pyspark.ml.regression import LinearRegression
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

In [2]:
spark = SparkSession.builder.appName('First_spark_LinearRegression_Project').getOrCreate()

In [3]:
cruisedata = spark.read.csv("cruise_ship_info.csv",inferSchema=True,header=True)
#Data types of the columns in the dataframe made using the data from CSV
cruisedata.printSchema()

root
 |-- Ship_name: string (nullable = true)
 |-- Cruise_line: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Tonnage: double (nullable = true)
 |-- passengers: double (nullable = true)
 |-- length: double (nullable = true)
 |-- cabins: double (nullable = true)
 |-- passenger_density: double (nullable = true)
 |-- crew: double (nullable = true)



In [4]:
#Data obtained from the CSV
cruisedata.show()

+-----------+-----------+---+------------------+----------+------+------+-----------------+----+
|  Ship_name|Cruise_line|Age|           Tonnage|passengers|length|cabins|passenger_density|crew|
+-----------+-----------+---+------------------+----------+------+------+-----------------+----+
|    Journey|    Azamara|  6|30.276999999999997|      6.94|  5.94|  3.55|            42.64|3.55|
|      Quest|    Azamara|  6|30.276999999999997|      6.94|  5.94|  3.55|            42.64|3.55|
|Celebration|   Carnival| 26|            47.262|     14.86|  7.22|  7.43|             31.8| 6.7|
|   Conquest|   Carnival| 11|             110.0|     29.74|  9.53| 14.88|            36.99|19.1|
|    Destiny|   Carnival| 17|           101.353|     26.42|  8.92| 13.21|            38.36|10.0|
|    Ecstasy|   Carnival| 22|            70.367|     20.52|  8.55|  10.2|            34.29| 9.2|
|    Elation|   Carnival| 15|            70.367|     20.52|  8.55|  10.2|            34.29| 9.2|
|    Fantasy|   Carnival| 23| 

In [5]:
#grouping the columns that can be used for predicting crew memebers
calculator = VectorAssembler(inputCols=["Age", "Tonnage", "passengers","length","cabins","passenger_density"],
                             outputCol="features")

In [6]:
outcome=calculator.transform(cruisedata)

In [7]:
#making a dataframe using the features and label column
total_data = outcome.select("features",'crew')

In [8]:
#splitting the dataframe into training and testing data
train_data,test_data = total_data.randomSplit([0.8,0.2])

In [9]:
#summary of training data
train_data.describe().show()

+-------+------------------+
|summary|              crew|
+-------+------------------+
|  count|               126|
|   mean| 7.755079365079375|
| stddev|3.5028488396804662|
|    min|              0.59|
|    max|              19.1|
+-------+------------------+



In [10]:
#summary of testing data
test_data.describe().show()

+-------+------------------+
|summary|              crew|
+-------+------------------+
|  count|                32|
|   mean| 7.948124999999998|
| stddev|3.5577670031056132|
|    min|              0.59|
|    max|              21.0|
+-------+------------------+



In [11]:
#defining LR model specifying the label
lr = LinearRegression(labelCol='crew')

In [12]:
#Trainign the LR with training data
lrModel = lr.fit(train_data,)

In [13]:
test_results = lrModel.evaluate(test_data)

In [14]:
test_results.residuals.show()

+--------------------+
|           residuals|
+--------------------+
|-0.00556834757052...|
|  0.4655397996926274|
| -0.7706746316692978|
|  -0.604742259334694|
| 0.47201445412662935|
|-0.36860040354372714|
| -0.5367110203027288|
| -0.4567702202114585|
|  0.2874534529224526|
| -0.9736715331866392|
| -1.4099036620120113|
|  0.4432420122929921|
|  0.7730489334849366|
| -1.5817831017951614|
|  0.7694422904314209|
| -1.4813782662918662|
|-0.02851111905918...|
|-0.02454967251743634|
|-0.07524225477451463|
|  0.5922002960912689|
+--------------------+
only showing top 20 rows



In [15]:
#making use of features to test the test data
unlabeled_data = test_data.select('features')

In [16]:
#Get the predicted values
predictions = lrModel.transform(unlabeled_data)

In [17]:
predictions.show()

+--------------------+------------------+
|            features|        prediction|
+--------------------+------------------+
|[4.0,220.0,54.0,1...|21.005568347570527|
|[5.0,122.0,28.5,1...| 6.234460200307373|
|[6.0,90.0,20.0,9....| 9.770674631669298|
|[8.0,110.0,29.74,...|12.204742259334694|
|[9.0,81.0,21.44,9...|  9.52798554587337|
|[9.0,116.0,26.0,9...|11.368600403543727|
|[10.0,81.76899999...| 8.956711020302729|
|[10.0,91.62700000...| 9.456770220211459|
|[11.0,58.6,15.66,...| 7.312546547077547|
|[11.0,85.0,18.48,...|  8.97367153318664|
|[11.0,138.0,31.14...|13.259903662012011|
|[12.0,42.0,14.8,7...| 6.356757987707008|
|[12.0,77.104,20.0...| 8.816951066515063|
|[13.0,63.0,14.4,7...| 6.891783101795161|
|[13.0,91.0,20.32,...|  9.22055770956858|
|[13.0,138.0,31.14...|13.241378266291866|
|[14.0,30.27699999...|3.7585111190591887|
|[14.0,30.27699999...|3.7545496725174363|
|[16.0,59.652,13.2...| 6.515242254774515|
|[16.0,77.71300000...| 8.497799703908731|
+--------------------+------------

In [18]:
test_data.show()

+--------------------+-----+
|            features| crew|
+--------------------+-----+
|[4.0,220.0,54.0,1...| 21.0|
|[5.0,122.0,28.5,1...|  6.7|
|[6.0,90.0,20.0,9....|  9.0|
|[8.0,110.0,29.74,...| 11.6|
|[9.0,81.0,21.44,9...| 10.0|
|[9.0,116.0,26.0,9...| 11.0|
|[10.0,81.76899999...| 8.42|
|[10.0,91.62700000...|  9.0|
|[11.0,58.6,15.66,...|  7.6|
|[11.0,85.0,18.48,...|  8.0|
|[11.0,138.0,31.14...|11.85|
|[12.0,42.0,14.8,7...|  6.8|
|[12.0,77.104,20.0...| 9.59|
|[13.0,63.0,14.4,7...| 5.31|
|[13.0,91.0,20.32,...| 9.99|
|[13.0,138.0,31.14...|11.76|
|[14.0,30.27699999...| 3.73|
|[14.0,30.27699999...| 3.73|
|[16.0,59.652,13.2...| 6.44|
|[16.0,77.71300000...| 9.09|
+--------------------+-----+
only showing top 20 rows



In [19]:
#comparing test labels with predictions
test_data.join(predictions,"features").select("crew","prediction").show()

+-----+------------------+
| crew|        prediction|
+-----+------------------+
| 21.0|21.005568347570527|
|  6.7| 6.234460200307373|
|  9.0| 9.770674631669298|
| 11.6|12.204742259334694|
| 10.0|  9.52798554587337|
| 11.0|11.368600403543727|
| 8.42| 8.956711020302729|
|  9.0| 9.456770220211459|
|  7.6| 7.312546547077547|
|  8.0|  8.97367153318664|
|11.85|13.259903662012011|
|  6.8| 6.356757987707008|
| 9.59| 8.816951066515063|
| 5.31| 6.891783101795161|
| 9.99|  9.22055770956858|
|11.76|13.241378266291866|
| 3.73|3.7585111190591887|
| 3.73|3.7545496725174363|
| 6.44| 6.515242254774515|
| 9.09| 8.497799703908731|
+-----+------------------+
only showing top 20 rows

